In [45]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas
import os
%matplotlib inline

In [46]:
# load image data
data_directory = '/Users/Psyche/neurohack2018/predictiveViz/data'
image_fname = os.path.join(data_directory, 'ICgroups.nii')
image = nib.load(image_fname)

In [47]:
print(image.shape)

(61, 73, 61)


In [48]:
# load coefficient data
coefs_fname = os.path.join(data_directory, 'coefficients.csv');
coefs = pandas.read_csv(coefs_fname);
# load masks
brainmask_fname = os.path.join(data_directory, 'MNI152maskbinary3mm.csv');
brainmask = pandas.read_csv(brainmask_fname, header=None); 
ROImask_fname = os.path.join(data_directory, 'cortical_mask_new.txt');
ROImask = pandas.read_csv(ROImask_fname, header=None);

In [49]:
print(brainmask.shape)
print(ROImask.shape)
print(coefs.shape[1])

(1, 271633)
(1, 271633)
120


In [60]:
# create a mask that is the intersection of the two
a = np.array(brainmask, dtype=bool)
b = np.array(ROImask, dtype=bool)
bigbrainROImask = (np.logical_and(a, b))*1
print(np.sum(bigbrainROImask==1))
bigbrainROImask

5252


array([[0, 0, 0, ..., 0, 0, 0]])

In [61]:
# create a 4D array -- a 3D array for each set of tuning parameters
data4D = np.zeros((int(image.shape[0]), int(image.shape[1]), int(image.shape[2]), coefs.shape[1]))
for i in range(coefs.shape[1]):
        data = bigbrainROImask.copy()
        data[bigbrainROImask==1] = coefs.iloc[:,i]*1000
        data4D[:,:,:,i] = np.reshape(data, image.shape, order='F')

In [62]:
# create nifti file from the 4D array
# copy the header and 
coef_header = image.header.copy()
coef_affine = image.affine.copy()

In [63]:
print(coef_header)

<class 'nibabel.nifti1.Nifti1Header'> object, endian='<'
sizeof_hdr      : 348
data_type       : b''
db_name         : b''
extents         : 0
session_error   : 0
regular         : b'r'
dim_info        : 0
dim             : [ 3 61 73 61  1  1  1  1]
intent_p1       : 0.0
intent_p2       : 0.0
intent_p3       : 0.0
intent_code     : none
datatype        : uint8
bitpix          : 8
slice_start     : 0
pixdim          : [-1.  3.  3.  3.  0.  0.  0.  0.]
vox_offset      : 0.0
scl_slope       : nan
scl_inter       : nan
slice_end       : 0
slice_code      : unknown
xyzt_units      : 10
cal_max         : 0.0
cal_min         : 0.0
slice_duration  : 0.0
toffset         : 0.0
glmax           : 0
glmin           : 0
descrip         : b''
aux_file        : b''
qform_code      : aligned
sform_code      : aligned
quatern_b       : 0.0
quatern_c       : 1.0
quatern_d       : 0.0
qoffset_x       : 90.0
qoffset_y       : -126.0
qoffset_z       : -72.0
srow_x          : [-3.  0.  0. 90.]
srow_y        

In [64]:
print(coef_affine)
print(data4D.dtype)
print()

[[  -3.    0.    0.   90.]
 [   0.    3.    0. -126.]
 [   0.    0.    3.  -72.]
 [   0.    0.    0.    1.]]
float64



In [65]:
# change data type to integer
coef_header.set_data_dtype(np.float32)

In [66]:
# create NiBabel nifti image
coef_image = nib.Nifti1Image(data4D, coef_affine, coef_header)

In [67]:
coef_image.to_filename(os.path.join(data_directory, 'coefs.nii'))